In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import matplotlib
from matplotlib.colors import ListedColormap

import os
import gc
import argparse
import torch
import optuna
import joblib
import pickle

from optuna.samplers import TPESampler
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

from collections import OrderedDict

from torch import nn
from torch.nn import Linear, ReLU, Sequential, LayerNorm
from torch.utils.data import DataLoader, TensorDataset
from torch.autograd import Variable

from torch_geometric.nn.models import GraphUNet
from torch_geometric.utils import dense_to_sparse, to_dense_adj

import geoad.nn.models as models
import geoad.utils.utils as utils
import geoad.utils.fault_detection as fd

from geoad.utils.utils import roc_params, compute_auc

from importlib import reload
models = reload(models)
utils = reload(utils)

from pyprojroot import here
root_dir = str(here())

data_dir = '~/data/interim/'

matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams.update({'font.family': 'DejaVu Serif'})

In [ ]:
# Toy example
torch.set_printoptions(precision=2)
torch.manual_seed(0)
 
device='cuda:1'

# Example usage:
N, T, F, r = 10, 12, 8, 2
X = torch.rand((N, T)).round(decimals=1)
X = X.to(device)

Sg = torch.randn((N,N))
Sg = Sg.matmul(Sg.t()).abs()
Sg = Sg.fill_diagonal_(0)
Sg = Sg/Sg.max()
# Sg[Sg<0.3] = 0
Sg = Sg.to(device)#.to_sparse()

# St = torch.eye(T).roll(1, dims=-1).t().to(device)
# ### St = torch.diag(torch.ones((T-1)), diagonal=-1)
# # St = torch.eye(T).roll(1, dims=-1).t()

# Use an optimizer (like SGD or Adam)
model = models.GTConvAE(hidden_features=[8,4], K=4, r=2, temp_graph='cyclic_directed', device=device, dense=True)
model = model.to(device)

model.reset_parameters()
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

loss_function = nn.MSELoss()

loss_values = []

n_steps = 200

for step in range(n_steps):
    print(model.s.detach().numpy().flatten())
    # Ensure the gradients are zero before starting optimizations
    optimizer.zero_grad()

    # Compute the output
    output = model(X, Sg)

    # Compute the loss as the difference between the output and the target
    loss = torch.nn.functional.mse_loss(output, X)
    loss_values.append(loss.cpu().detach())
    # Backpropagate the error
    loss.backward()

    # Update h0
    optimizer.step()

plt.plot(loss_values)
plt.show()

output
    

### TEST MODEL

-----------------------------

In [ ]:
# rng_seed = 0
# torch.manual_seed(rng_seed)
# torch.cuda.manual_seed(rng_seed)
# np.random.seed(rng_seed)

use_weight = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = 'cuda:1'

np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)

dataset = 'df_StOlavs_D1L2B'
df_orig = pd.read_parquet(data_dir + f'{dataset}.parq')

df_ds = df_orig[df_orig.timestamp<'2022-06'].copy()
df_ds = df_ds.groupby('pid').resample('30d', on='timestamp').mean().reset_index()

data, labels, data_dfs, G, nodes = utils.generate_data(df_ds, 10, 0, 1, anomalous_nodes=20)

# Weighted adjacency matrix
A = torch.tensor(G.W.toarray()).float().to(device)

G.compute_laplacian('normalized')
Ls = torch.tensor(G.L.toarray()).float().to(device) # symmetric normalized Laplacian

# Possible hyperparameters
hidden_features = [8, 4]
K=6
r=2


N_epochs = 50
batch_size = 2048

model = models.GTConvAE(hidden_features=hidden_features, K=K, r=r, temp_graph='cyclic_directed', device=device, dense=True)
model = model.to(device)

loss_function = nn.MSELoss()

for i in range(data.shape[0]):
    print(i)
    X = torch.tensor(data[i,:,:]).float().to(device)
    norm_X = LayerNorm(X.shape, elementwise_affine=False) # Normalizes the <entire matrix> to 0 mean 1 var
    X = norm_X(X)
    
    label = labels[i,:]

    loss_values = []

    model.train()
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in tqdm(range(N_epochs)):
         
        optimizer.zero_grad()
        output = model(X, Ls)
        loss = torch.nn.functional.mse_loss(X, output)
        loss_values.append(loss.cpu().detach())
        loss.backward()
        optimizer.step()

plt.plot(loss_values)
plt.show()

model.eval()
Y = model(X, Ls)

f = nn.MSELoss(reduction='none')
score = torch.mean(f(X,Y), axis=1).cpu().detach().numpy()

nodes['anomaly'] = label
nodes['score'] = score

tpr, fpr, _ = roc_params(metric=score, label=label, interp=True)
auc = compute_auc(tpr,fpr)
auc

In [ ]:
label_cmap = ListedColormap(plt.cm.viridis(np.linspace(0,1,2)))

fig, ax = plt.subplots(ncols=2, figsize=(16,5))
plotting_params = {'edge_color':'darkgray', 'edge_width':1.5,'vertex_color':'black', 'vertex_size':50}
G.plotting.update(plotting_params)
G.plot_signal(label, ax=ax[0], plot_name='Label')

ax[0].collections[0].set_cmap(label_cmap)  # Modify the colormap of the plotted data
ax[0].axis('off')

G.plot_signal(score, ax=ax[1], plot_name='Anomaly Score')
ax[1].collections[0].set_cmap('viridis')
ax[1].axis('off')

plt.show()

In [ ]:
utils.visualize_map(nodes, color='anomaly', size=np.ones(nodes.pid.nunique()), size_max=10, title='Label',
                     hover_data=['cluster'], zoom=15, figsize=(600,600), colormap='viridis')
                    
utils.visualize_map(nodes, color='score', size=np.ones(nodes.pid.nunique()), size_max=10, title='Anomaly score',
                    hover_data=['cluster'], zoom=15, figsize=(600,600), colormap='viridis')